# Fully Connected Network

This is the second part of our whole deep learning planning structure. 

The Data in this experiments is generated from RDDL simulator [Github](https://github.com/ssanner/rddlsim), which is written by Prof.Scott Sanner at University of Toronto.

In the section, we need a fully connected network to compute the reward of each (STATE,ACTION,STATE') tuple, that is (STATE,ACTION,STATE') -> Reward. Since this part is deterministic, fully connected network is capable to solve.

Problem list:
1. Data normalization will highly impact the network performance, we need to normalize the input. However, the input of this section is an output of VAE, which is unnormalized. And since everything is working under tensorflow environment, we need to build normalizer inside tensorflow graph.
2. For reward function R(s,a), in nondeterministic domain, R(s,a) is stochastic. We need a deterministic function. Therefore, we rewrite the reward function as R(s,a,s'). This requires us to concate s,a,s' as single input matrix(also under tensorflow).

### Import Packages
We do note provide pip installation commands, please search this package and install it through pip install. Please upgrade your pip before installing, since old pip would cause errors.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial
import re

### Load Data
We load data from csv files. The following code shows how to load data through pandas and numpy. Result of this progress can be feed into tensorflow with "feed_dict" argument. 

In [2]:
#Data Path..
Datapath="DATA/Navigation/Nav_RDDL_Data.txt"
Labelpath="DATA/Navigation/Nav_RDDL_Label.txt"
Rewardpath="DATA/Navigation/Nav_RDDL_Reward.txt"

In [3]:
#Given local path, find full path
def PathFinder(path):
    #python 2
    #script_dir = os.path.dirname('__file__')
    #fullpath = os.path.join(script_dir,path)
    #python 3
    fullpath=os.path.abspath(path)
    print(fullpath)
    return fullpath

#Read Data for Deep Learning
def ReadData(path):
    fullpath=PathFinder(path)
    return pd.read_csv(fullpath, sep=',', header=0)

#Won't use this one to normalize
#Input Normalization
def Normalize(features, mean = [], std = []):
    if mean == []:
        mean = np.mean(features, axis = 0)
        std = np.std(features, axis = 0)
#     print std
#     print std[:,None]
    new_feature = (features.T - mean[:,None]).T
    new_feature = (new_feature.T / std[:,None]).T
    new_feature[np.isnan(new_feature)]=0
#     print new_feature
    return new_feature, mean, std

In [4]:
x_pd = ReadData(Datapath)
y_pd = ReadData(Labelpath)
r_pd = ReadData(Rewardpath)

/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Nav_RDDL_Data.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Nav_RDDL_Label.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Navigation/Nav_RDDL_Reward.txt


In [5]:
#Get Numpy Arrays
x_matrix=x_pd.as_matrix()
y_matrix=y_pd.as_matrix()
r_matrix=r_pd.as_matrix()

In [6]:
data_size=len(x_matrix)
# Uppercase for constants
INPUT_S_A_SIZE = 4
INPUT_SP_SIZE = 2
OUTPUT_SIZE = 1

### Support Functions
The following functions allows us passthrough multiple functions without explicitly assign intermediate output variables.

In [7]:
def compose(f,g):
    return lambda x:g(f(x))
    
def composeAll(*args):
    """
    composeAll([f,g,h])(x): f(g(h(x)))
    """
    return partial(functools.reduce, compose)(*args)

## Tensorflow
### Input tensor place holders

In [8]:
# Input features s,a
x = tf.placeholder(tf.float32,[None, INPUT_S_A_SIZE],name="Features_S_A")

# Input features s'
y = tf.placeholder(tf.float32, [None, INPUT_SP_SIZE],name="Features_SP")

# Input label
r = tf.placeholder(tf.float32, [None, OUTPUT_SIZE],name="Labels")

### Variable Generating functions

In [9]:
#Weight constructing function
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.001)
    return tf.Variable(initial,name="Matrix")

#Bias constructing function
def bias_variable(shape):
    initial = tf.constant(0.,shape=shape)
    return tf.Variable(initial,name="Bias")

### Fully Connected Layer Defination

In [10]:
class Dense():
    """Fully Connected Layer"""
    def __init__(self, scope="fully_connected_layer", output_dim =None, dropout=1.0, activation=tf.identity):
        assert output_dim, "Missing output dimension specification!"
        self.scope = scope
        self.output_dim = output_dim
        self.dropout = dropout
        self.activation = activation
        
    def __call__(self,x):
        with tf.name_scope(self.scope):
            while True:
                try:
                    return self.activation(tf.matmul(x,self.w)+self.b)
                except(AttributeError):
                    self.w = tf.nn.dropout(weight_variable([x.get_shape()[1].value, self.output_dim]),self.dropout)
                    self.b = bias_variable([self.output_dim])
    
    def set_parameters(self, weight, bias):
        self.w.assign(weight)
        self.b.assign(bias)
        
    def get_l2_loss(self):
        return tf.nn.l2_loss(self.w)

### Full Deep Network Class
The following class define a complete deep network, which include:
1. Network structure specification
2. Loss function specification
3. Prediction specification
4. Optimization method specification
5. Training function
6. Saving function(not tensorflow variable saving, but numpy weight dumping!)
7. Loading function(not tensorflow variable loading, but numpy weight assignment!)
8. Mini-Batch generation function

In [11]:
class DeepNet(object):
    
    def __init__(self, 
                 x, #Input Features for S,A
                 y, #Input Features for S'
                 r, #Output Label for R
                 num_hidden_layers, #number of layers for both encoder and decoder
                 num_hidden_nodes, #number of nodes in each layer
                 activation, #nonlinear activation function
                 learning_rate=0.001, #Learning rate
                 batch_size=100, 
                 dropout = 1.0,
                 l2_lambda = 0): #Batch size        
        self.mean = tf.Variable(tf.zeros([x.get_shape()[1]+y.get_shape()[1]]),trainable=False,name="NORM_MEAN")
        self.var = tf.Variable(tf.ones([x.get_shape()[1]+y.get_shape()[1]]),trainable=False,name="NORM_VAR")
        self.f = self._p_normalize(self._p_concate(x,y))
        self.r = r
        self.num_hidden_layers = num_hidden_layers
        self.num_hidden_nodes = num_hidden_nodes
        self.activation = activation
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.l2_lambda = l2_lambda
        self.dropout = dropout
        self._p_create_dnn_graph()
        self._p_create_loss()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        
    def _p_create_dnn_graph(self):

        layers = []
        for i in range(self.num_hidden_layers):
            layers.append(Dense("layer"+str(i),self.num_hidden_nodes,self.dropout))
        layers.append(Dense("layer"+str(self.num_hidden_layers),self.r.get_shape()[1].value,self.dropout))
        self.r_pred = composeAll(layers)(self.f)
        self.layers = layers 
    
    def _p_concate(self,x,y):
        return tf.concat(1,[x,y])
    
    def _p_normalize(self, unnormed):
        epsilon = 1e-3
        normed = tf.nn.batch_normalization(unnormed,self.mean,self.var,None,None,epsilon)
        return normed
        
    def _p_create_loss(self): #lambda for l2 regularization

        #L2 regularization loss
        l2_loss = tf.constant(0.0)
        for layer in self.layers:
            l2_loss += layer.get_l2_loss()

        #Mean Squared Error
        mse_r = tf.reduce_mean(tf.square(tf.sub(self.r,self.r_pred)), reduction_indices=1)

        #loss
        self.loss = tf.reduce_mean(mse_r)+self.l2_lambda*l2_loss
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)   
        
    def update_normalization(self,x_matrix,y_matrix):
        tf_mean,tf_var = tf.nn.moments(self._p_concate(x,y), axes = [0])
        feed_dict = {x:x_matrix,y:y_matrix}
        np_mean,np_var = self.sess.run([tf_mean,tf_var],feed_dict=feed_dict)
        print(np_mean)
        self.sess.run(self.mean.assign(np_mean))
        self.sess.run(self.var.assign(np_var))
        print(self.mean.eval())
        
    
    def train_model(self,data_s_a,data_sp,data_r,epoch=100):
        
        self.update_normalization(data_s_a,data_sp)
        
        batches = self._p_get_batches(data_s_a,data_sp,data_r,self.batch_size)
        
        self.mean = tf.Variable(tf.zeros([x.get_shape()[1]+y.get_shape()[1]]),trainable=False)
        self.var = tf.Variable(tf.ones([x.get_shape()[1]+y.get_shape()[1]]),trainable=False)
        
        summary_writer = tf.summary.FileWriter('experiment', graph=self.sess.graph)

        #Training
        for epoch in range(epoch):
            for step in range(len(batches)):
                feed_dict = {x: batches[step][0],y: batches[step][1],r: batches[step][2]}
                training = self.sess.run([self.optimizer], feed_dict=feed_dict)
            new_loss = self.sess.run([self.loss],feed_dict=feed_dict)
            print('Loss in epoch {0}: {1}'.format(epoch, new_loss))    
            
    def _p_get_batches(self,x_matrix,y_matrix,r_matrix,batch_size):
        remaining_size = len(x_matrix)
        batch_index=0
        batches = []
        while(remaining_size>0):
            batch = []
            if remaining_size<batch_size:
                batch.append(x_matrix[batch_index*batch_size:-1])
                batch.append(y_matrix[batch_index*batch_size:-1])
                batch.append(r_matrix[batch_index*batch_size:-1])
            else:
                batch.append(x_matrix[batch_index*batch_size:(batch_index+1)*batch_size])
                batch.append(y_matrix[batch_index*batch_size:(batch_index+1)*batch_size]) 
                batch.append(r_matrix[batch_index*batch_size:(batch_index+1)*batch_size]) 
            batch_index+=1
            remaining_size-=batch_size
            batches.append(batch)
        return batches
    
    def _p_extract_weights(self):
        # a hashmap maps from layer name to weights and biases
        mp_layer_weights = {}

        #iteratively save values
        for dense in self.layers:
            values = {'weights':dense.w, 'biases':dense.b}
            mp_layer_weights[layer.scope] = values
        
        norms = {'mean':self.mean, 'var':self.var}
        mp_layer_weights['normalizations'] = norms

        return mp_layer_weights
    
    def save_weights(self,path):
        #extract weights from trained model
        layer_weights = self.sess.run(_p_extract_weights())
        print('Whole layer weights: {0}'.format(layer_weights))
        np.save(path,layer_weights)
    
    def load_weights(self,path):
        layer_weights = np.load(path)
        for dense in self.layers:
            print('Scope:{0}'.format(dense.scope))
            values = layer_weights.get(dense.scope)
            weights = values.get('weights')
            biases = values.get('biases')
            dense.set_parameters(weights,biases)
        print('Done!')
        
    def save_variables_for_rnn(self,path,prefix="RNN/FullNetworkCell/Reward/"):
        variables = tf.trainable_variables()
        var_dict = {}
        for v in variables:
            if v.name.contains("/read"):
                name = prefix+re.sub("/read", "", v.name)
                var_dict["name"] = v
        for k in var_dict.keys:
            print(k)
        saver = tf.train.Saver(var_dict)
        saver.save(self.sess, PathFinder(path))     

In [12]:
#Instantiate a network
dnn_inst = DeepNet(x,y,r,2,20,tf.nn.relu)

### Tensorflow graph visualization function

In [13]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:960px;height:600px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [14]:
show_graph(tf.get_default_graph().as_graph_def())

### Training

In [15]:
dnn_inst.train_model(x_matrix,y_matrix,r_matrix,100)

[ 0.31160992  0.31038597  1.82958364  1.82337689  2.13955069  2.13230014]
[ 0.31160992  0.31038597  1.82958364  1.82337689  2.13955069  2.13230014]
Loss in epoch 0: [0.0033714061]
Loss in epoch 1: [0.0011653679]
Loss in epoch 2: [0.00035777671]
Loss in epoch 3: [0.00067403284]
Loss in epoch 4: [0.00050032861]
Loss in epoch 5: [0.00042203499]
Loss in epoch 6: [0.00043650973]
Loss in epoch 7: [0.00051351194]
Loss in epoch 8: [0.00055498385]
Loss in epoch 9: [0.00058958557]
Loss in epoch 10: [0.00060639309]
Loss in epoch 11: [0.00061609113]
Loss in epoch 12: [0.00062328833]
Loss in epoch 13: [0.00062916952]
Loss in epoch 14: [0.00063368061]
Loss in epoch 15: [0.00063975691]
Loss in epoch 16: [0.000651446]
Loss in epoch 17: [0.00064611534]
Loss in epoch 18: [0.0006580053]
Loss in epoch 19: [0.0006594829]
Loss in epoch 20: [0.00066607597]
Loss in epoch 21: [0.00067935331]
Loss in epoch 22: [0.00067139021]
Loss in epoch 23: [0.00079808303]
Loss in epoch 24: [0.00066099159]
Loss in epoch 25: 

In [16]:
#Saving function checking..
dnn_inst.layers[0].w

<tf.Tensor 'layer0/Matrix/read:0' shape=(6, 20) dtype=float32>

In [17]:
tf.trainable_variables()